<a href="https://colab.research.google.com/github/thai94/d2l/blob/main/5.deep_learning_computation/5_2_parameter_management_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
import torch
from torch import nn

In [91]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.5542],
        [0.4852]], grad_fn=<AddmmBackward0>)

In [92]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.3390,  0.2546,  0.2728,  0.0550, -0.2716, -0.1370,  0.1160, -0.2671]])), ('bias', tensor([0.3124]))])


In [93]:
type(net[2].bias)

torch.nn.parameter.Parameter

In [94]:
net[2].bias

Parameter containing:
tensor([0.3124], requires_grad=True)

In [95]:
net[2].bias.data

tensor([0.3124])

In [96]:
net[2].weight.grad == None

True

In [97]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))


In [98]:
print(*[(name, param.shape) for name, param in net.named_parameters()])

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [99]:
net.state_dict()['2.bias']

tensor([0.3124])

In [100]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())
    
def block2():
    net = nn.Sequential()
    for i in range(4):
        # Nested here
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.3666],
        [0.3665]], grad_fn=<AddmmBackward0>)

In [101]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [102]:
print(rgnet[0][1][0].bias.data)

tensor([ 0.3619, -0.0407, -0.0843, -0.2109,  0.4347,  0.1452, -0.0060,  0.0586])


In [103]:
def init_normal(m):
  if type(m) == nn.Linear:
    nn.init.normal_(m.weight, mean = 0, std = 0.01)
    nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0145, -0.0118,  0.0106, -0.0095]), tensor(0.))

In [104]:
def init_constant(m):
  if type(m) == nn.Linear:
     nn.init.constant_(m.weight, 1)
     nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [105]:
def xavier(m):
  if type(m) == nn.Linear:
    nn.init.xavier_uniform_(m.weight)

def init_42(m):
  if type(m) == nn.Linear:
    nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)

print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.5382, -0.4232, -0.2033, -0.6542])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [107]:
def my_init(m):
  
  if type(m) == nn.Linear:
    nn.init.uniform_(m.weight, -10, 10)
    m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight

Parameter containing:
tensor([[-0.0000,  8.8338, -8.6878,  5.9086],
        [ 7.4437,  8.3120,  0.0000,  0.0000],
        [ 5.1359, -0.0000,  0.0000,  6.8235],
        [-5.4196, -5.5619,  0.0000, -0.0000],
        [ 0.0000, -0.0000, -6.4601,  6.2168],
        [-0.0000,  0.0000, -8.1438, -8.2487],
        [-8.1329,  0.0000, -5.3540, -7.6433],
        [-0.0000, -0.0000, -0.0000,  6.5549]], requires_grad=True)

In [111]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))

net(X)

print(net[2].weight.data[0] == net[4].weight.data[0])

net[2].weight.data[0, 0] = 100

print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [ ]:
y = net(X)
y.sum().backward()

